# Valuing European-Style Swaptions Across Models

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.rates import *
from financepy.finutils import *
from financepy.products.rates.FinIborSingleCurve import FinIborSingleCurve
from financepy.market.curves.FinInterpolator import FinInterpTypes

####################################################################
# FINANCEPY BETA Version 0.185 - This build:  24 Oct 2020 at 21:27 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Building a Libor Discount Curve

In [4]:
valuation_date = FinDate(28, 2, 2014)
settlement_date = FinDate(4, 3, 2014)

We have a vector of dates and zero rates

In [5]:
depoDCCType = FinDayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = FinIborDeposit(settlement_date, "1W", 0.0023, depoDCCType); depos.append(depo)
depo = FinIborDeposit(settlement_date, "1M", 0.0023, depoDCCType); depos.append(depo)
depo = FinIborDeposit(settlement_date, "3M", 0.0023, depoDCCType); depos.append(depo)
depo = FinIborDeposit(settlement_date, "6M", 0.0023, depoDCCType); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
swapType = FinSwapTypes.PAY
fixedDCCType = FinDayCountTypes.ACT_365F
fixedFreqType = FinFrequencyTypes.SEMI_ANNUAL

swap = FinIborSwap(settlement_date, "3Y", swapType, 0.00790, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "4Y", swapType, 0.01200, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "5Y", swapType, 0.01570, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "6Y", swapType, 0.01865, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "7Y", swapType, 0.02160, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "8Y", swapType, 0.02350, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "9Y", swapType, 0.02540, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "10Y", swapType, 0.0273, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "15Y", swapType, 0.0297, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "20Y", swapType, 0.0316, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "25Y", swapType, 0.0335, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "30Y", swapType, 0.0354, fixedFreqType, fixedDCCType); swaps.append(swap)

libor_curve = FinIborSingleCurve(valuation_date, depos, [], swaps)

Inserting synthetic deposit


In [6]:
exerciseDate = settlement_date.addTenor("5Y")
swapMaturityDate = exerciseDate.addTenor("5Y")
swapFixedCoupon = 0.040852
swapFixedFrequencyType = FinFrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = FinDayCountTypes.THIRTY_E_360_ISDA
swapFloatFrequencyType = FinFrequencyTypes.QUARTERLY
swapFloatDayCountType = FinDayCountTypes.ACT_360
swapNotional = ONE_MILLION
swapType = FinSwapTypes.PAY  
calendar_type = FinCalendarTypes.WEEKEND
bus_day_adjust_type = FinBusDayAdjustTypes.NONE
date_gen_rule_type = FinDateGenRuleTypes.BACKWARD

In [7]:
swaption = FinIborSwaption(settlement_date,
                            exerciseDate,
                            swapMaturityDate,
                            swapType,
                            swapFixedCoupon,
                            swapFixedFrequencyType,
                            swapFixedDayCountType, 
                            swapNotional, 
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            calendar_type, 
                            bus_day_adjust_type,
                            date_gen_rule_type)

## Valuation using Black's Model

In [8]:
model = FinModelBlack(0.1533)

In [9]:
swaption.value(settlement_date, libor_curve, model)

23163.334872028132

## Valuation using Shifted Black

In [22]:
model = FinModelBlackShifted(0.1533, 0.008)

In [23]:
swaption.value(settlement_date, libor_curve, model)

27695.460283579072

##  Valuation using SABR

In [12]:
model = FinModelSABR(0.132, 0.5, 0.5, 0.5)

In [13]:
swaption.value(settlement_date, libor_curve, model)

104583.52188346579

## Valuation using Shifted SABR

In [14]:
model = FinModelSABRShifted(0.1, 0.2, 0.1, 0.15, -0.005)

In [15]:
swaption.value(settlement_date, libor_curve, model)

163445.66166978356

## Valuation using Hull-White

In [16]:
model = FinModelRatesHW(0.01, 0.02)

In [17]:
swaption.value(settlement_date, libor_curve, model)

34173.86962564415

## Valuation using Black-Karasinski

In [18]:
model = FinModelRatesBK(0.2, 0.05)

In [19]:
swaption.value(settlement_date, libor_curve, model)

23474.88575736677

## Internals

In [20]:
print(swaption)

OBJECT TYPE: FinIborSwaption
SETTLEMENT DATE: TUE 04 MAR 2014
EXERCISE DATE: MON 04 MAR 2019
SWAP TYPE: FinSwapTypes.PAY
SWAP MATURITY DATE: MON 04 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FinFrequencyTypes.QUARTERLY
FLOAT DAY COUNT: FinDayCountTypes.ACT_360
PV01: 4.161673891171295
FWD SWAP RATE: 4.086047153125578
FWD DF TO EXPIRY: 0.9237697293451264


We can see that the forward swap rate almost equals the fixed coupon. The underlying swap is close to being ATM forward.

In [21]:
swaption.printSwapFixedLeg()

START DATE: MON 04 MAR 2019
MATURITY DATE: MON 04 MAR 2024
COUPON (%): 4.0852
FIXED LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE TUE 04 MAR 2014
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
TUE 04 MAR 2014          -            -   1.00000000            -            -
WED 04 SEP 2019  0.5000000     20426.00   0.90872318     18561.58     18561.58
WED 04 MAR 2020  0.5000000     20426.00   0.89262979     18232.86     36794.44
FRI 04 SEP 2020  0.5000000     20426.00   0.87518398     17876.51     54670.94
THU 04 MAR 2021  0.5000000     20426.00   0.85693776     17503.81     72174.75
SAT 04 SEP 2021  0.5000000     20426.00   0.84102606     17178.80     89353.55
FRI 04 MAR 2022  0.5000000     20426.00   0.82477877     16846.93    106200.48
SUN 04 SEP 2022  0.5000000     20426.00   0.80771756     16498.44    122698.92
SAT 04 MAR 2023  0.5000000     20426.00   0.79039583     16144.63    1388

Copyright (c) 2020 Dominic O'Kane